# Kanren Reasoner
miniKanren-based Inference Engine for Non-Axiomatic Logic

#### Dependencies

In [ ]:
%pip install lark
%pip install pyyaml
%pip install miniKanren

import os
import yaml

# download required files
if not os.path.isfile('Narsese/rules.yml'):
  !mkdir Narsese
  !wget -P Narsese https://raw.githubusercontent.com/maxeeem/nars-workshop-2024/main/Narsese/rules.yml
  !wget -P Narsese https://raw.githubusercontent.com/maxeeem/nars-workshop-2024/main/Narsese/Narsese.py
  !wget -P Narsese https://raw.githubusercontent.com/maxeeem/nars-workshop-2024/main/Narsese/narsese.lark
  !wget -P Narsese https://raw.githubusercontent.com/maxeeem/nars-workshop-2024/main/Narsese/narsese_lark.py
  !wget -P Narsese https://raw.githubusercontent.com/maxeeem/nars-workshop-2024/main/Narsese/parser.py

from Narsese.Narsese import *
from Narsese.parser import parse
from cons import cons, car, cdr
from kanren import var, run, eq

## Parser
Uses grammar defined in `narsese.lark`

#### String → Narsese

In [2]:
# generate narsese_lark.py from grammar definition in narsese.lark
os.system(f'python -m lark.tools.standalone ./Narsese/narsese.lark > ./Narsese/narsese_lark.py')

task = parse("<dog --> animal>.")

print('Judgement:', task, '\n')

statement = task.term

print('Subject:', statement.subject)
print('Copula:', statement.copula)
print('Predicate:', statement.predicate)

print('\nNarsese:', statement)

Judgement: <dog --> animal>. %1.00;0.90% 

Subject: dog
Copula: -->
Predicate: animal

Narsese: <dog --> animal>


#### Narsese → Logic

In [3]:
def logic(term: Term, rule=False):
    if term.is_atom:
        return var(term) if rule else term
    if term.is_statement:
        subject = logic(term.subject, rule)
        copula = term.copula
        predicate = logic(term.predicate, rule)
        return cons(copula, subject, predicate)
    
print('Logic:', logic(statement))

Logic: (--> . (dog . animal))


#### Logic → Narsese

In [4]:
def term(logic):
    if type(logic) is Term:
        return logic
    if type(logic) is cons:
        subject = car(cdr(logic))
        copula = car(logic)
        predicate = cdr(cdr(logic))
        return Statement(term(subject), copula, term(predicate))
    return logic # var

print('Narsese:', term(logic(statement)))

Narsese: <dog --> animal>


## Rules
Stored in a human-readable format in `rules.yml`

#### Load rules from a file

In [5]:
def convert(rule):
    # split rule string into components
    # ex: {<M --> P>. <S --> M>} |- <S --> P> .ded
    
    # p1: <M --> P>, p2: <S --> M>
    # c: <S --> P>, r: ded
    
    components = rule.split(" |- ")
    p1, p2 = components[0].strip("{}").split(". ")
    c, r = components[1].split(" .")

    # convert string to narsese 
    p1 = parse(p1 + '.', True)
    p2 = parse(p2 + '.', True)
    c  = parse(c  + '.', True)

    # convert narsese to logic
    p1 = logic(p1, True)
    p2 = logic(p2, True)
    c  = logic(c , True)

    return (p1, p2, c, r)

truth_functions = {
    'ded': Truth_deduction,
    'abd': Truth_abduction,
    'ind': Truth_induction,
    'exe': Truth_exemplification,
    'ana': Truth_analogy,
    'com': Truth_comparison,
    'res': Truth_resemblance
}

with open('Narsese/rules.yml', 'r') as file:
    config = yaml.safe_load(file)
    
nal1 = config['rules']['nal1'].splitlines()
nal2 = config['rules']['nal2'].splitlines()

rules = [convert(rule) for rule in nal1 + nal2]

#### Sample

In [6]:
def print_rule(rule):
    (p1, p2, c, r) = rule
    print(r.upper(), 'rule')
    print('Premise 1 ', p1)
    print('Premise 2 ', p2)
    print('Conclusion', c)

for i in range(3):
    print('\n', nal1[i], '\n')
    print_rule(rules[i])


 {<M --> P>. <S --> M>} |- <S --> P> .ded 

DED rule
Premise 1  (--> . (~M . ~P))
Premise 2  (--> . (~S . ~M))
Conclusion (--> . (~S . ~P))

 {<M --> P>. <M --> S>} |- <S --> P> .ind 

IND rule
Premise 1  (--> . (~M . ~P))
Premise 2  (--> . (~M . ~S))
Conclusion (--> . (~S . ~P))

 {<P --> M>. <S --> M>} |- <S --> P> .abd 

ABD rule
Premise 1  (--> . (~P . ~M))
Premise 2  (--> . (~S . ~M))
Conclusion (--> . (~S . ~P))


## Inference
Applying rules using miniKanren

In [7]:
def inference(t1: Sentence, t2: Sentence) -> list:
    results = []

    l1 = logic(t1.term)
    l2 = logic(t2.term)

    for rule in rules:
        (p1, p2, c, r) = rule
    
        result = run(1, c, eq((p1, p2), (l1, l2)))
        
        if result:
            conclusion = term(result[0])
            truth = truth_functions[r](t1.truth, t2.truth)
            results.append(((conclusion, r), truth))

    return results

## Examples

In [8]:
t1 = parse("<dog --> animal>.")
t2 = parse("<Sandy --> dog>.")

inference(t1, t2)

[((Statement: <Sandy --> animal>, 'ded'), %1.00;0.81%)]

In [9]:
t1 = parse("<Sandy --> dog>.")
t2 = parse("<dog <-> pet>. %0.80%")

inference(t1, t2)

[((Statement: <Sandy --> pet>, 'ana'), %0.80;0.65%)]